In [1]:
import pandas as pd

聚合运算
=====

在数据分析中，对数据聚合（求和、平均值等）通常是不可避免的。在将数据分组（groupby）后，新手若不知函数 agg() 的情况下，很有可能写出二三十行的for循环只为求每个分组的平均值。在这里，介绍个强大的函数：agg() ,能让你的代码缩减到一行

In [2]:
df1 = pd.DataFrame({'sex':list('FFMFMMF'),'smoker':list('YNYYNYY'),'age':[21,30,17,37,40,18,26],'weight':[120,100,132,140,94,89,123]})
print(df1)

  sex smoker  age  weight
0   F      Y   21     120
1   F      N   30     100
2   M      Y   17     132
3   F      Y   37     140
4   M      N   40      94
5   M      Y   18      89
6   F      Y   26     123


In [3]:
grouped=df1.groupby(['sex','smoker'])
print(grouped)

#求单列下聚合
print(grouped['age'].agg('mean'))


#求多列下聚合
print(grouped.agg('mean'))

# 求多种聚合运算
print(grouped['age'].agg(['min','max']))

#求多种聚合运算的同时更改列名
print(grouped['age'].agg([('A','mean'),('B','max')]))

#不同的列运用不同的聚合函数
print(grouped.agg({'age':['sum','mean'],'weight':['min','max']}))



sex  smoker
F    N         30.0
     Y         28.0
M    N         40.0
     Y         17.5
Name: age, dtype: float64
             age      weight
sex smoker                  
F   N       30.0  100.000000
    Y       28.0  127.666667
M   N       40.0   94.000000
    Y       17.5  110.500000
            min  max
sex smoker          
F   N        30   30
    Y        21   37
M   N        40   40
    Y        17   18
               A   B
sex smoker          
F   N       30.0  30
    Y       28.0  37
M   N       40.0  40
    Y       17.5  18
           age       weight     
           sum  mean    min  max
sex smoker                      
F   N       30  30.0    100  100
    Y       84  28.0    120  140
M   N       40  40.0     94   94
    Y       35  17.5     89  132


In [4]:
#使用自定义的聚合函数
def Max_cut_Min(group):
    return group.max()-group.min()

print(grouped.agg(Max_cut_Min))
print(grouped.describe())

            age  weight
sex smoker             
F   N         0       0
    Y        16      20
M   N         0       0
    Y         1      43
             age                                                 weight  \
           count  mean       std   min    25%   50%    75%   max  count   
sex smoker                                                                
F   N        1.0  30.0       NaN  30.0  30.00  30.0  30.00  30.0    1.0   
    Y        3.0  28.0  8.185353  21.0  23.50  26.0  31.50  37.0    3.0   
M   N        1.0  40.0       NaN  40.0  40.00  40.0  40.00  40.0    1.0   
    Y        2.0  17.5  0.707107  17.0  17.25  17.5  17.75  18.0    2.0   

                                                                        
                  mean        std    min     25%    50%     75%    max  
sex smoker                                                              
F   N       100.000000        NaN  100.0  100.00  100.0  100.00  100.0  
    Y       127.666667  10.785793  120

Apply–对行、列用函数处理
======
apply函数，可以说是pandas中自由度最高的函数。不过，是否能发挥其巨大威力，取决于我们的创造力。

首先我们必须弄清楚，之前都是对列进行操作，而现在是求每一行，也就是每个人的含水量，即对行进行操作，故应用apply函数时axis=1。而每一次应用自定义的water函数时，传进的一行是Series对象，每次返回单值，最终组合成Series对象。

## 总结
首先，我们要弄清操作的元素是行还是列，若对每一行进行apply，则axis=1。其次，编写好apply应用的函数，注意传进的参数和返回值的类型是单值、Series或DataFrame。最后，groupby 和 apply 组合，运用得当将会产生巨大的威力！


In [5]:
df1=pd.DataFrame({'sex':list('FFMFMMF'),'smoker':list('YNYYNYY'),'age':[21,30,17,37,40,18,26],'weight':[120,100,132,140,94,89,123]})

print(df1)

  sex smoker  age  weight
0   F      Y   21     120
1   F      N   30     100
2   M      Y   17     132
3   F      Y   37     140
4   M      N   40      94
5   M      Y   18      89
6   F      Y   26     123


In [6]:
def bin_age(age):
    if age >=18:
        return 1
    else:
        return 0

df1['age'].apply(bin_age)
df1['age'].apply(lambda x : 1 if x >18 else 0)

def water(one_row):
    #婴儿
    if one_row['age']==1:
        return one_row['weight']*0.7
    #成年男女
    if one_row['sex']=='M':
        return one_row['weight']*0.6
    else:
        return one_row['weight']*0.5

print(df1.apply(water,axis=1))


0    60.0
1    50.0
2    79.2
3    70.0
4    56.4
5    53.4
6    61.5
dtype: float64
